In [15]:
## import required libraries
import networkx as nx
import numpy as np
import random
from itertools import combinations
import json

# Part 2: Email behaviour data

## Task 1 of 2

**Q1. Do the majority of individuals have a higher or lower ratio of mutual connections than average in the network?**

In [5]:
# Load the email behavior data as a directed graph
email_graph = nx.read_edgelist("emails_cmt224 (1).edgelist", nodetype=int, create_using=nx.DiGraph())

# Calculate the mutual connections ratio for each individual
mutual_connections_ratios = {}
for node in email_graph.nodes():
    # Get successors (individuals the current individual has sent emails to)
    successors = set(email_graph.successors(node))
    mutual_connections = 0
    # Check for mutual connections
    for successor in successors:
        if email_graph.has_edge(successor, node):
            mutual_connections += 1
    # Calculate ratio
    total_connections = len(successors)
    mutual_connections_ratio = mutual_connections / total_connections if total_connections != 0 else 0
    mutual_connections_ratios[node] = mutual_connections_ratio

# Calculate the average ratio across all individuals
average_ratio = sum(mutual_connections_ratios.values()) / len(mutual_connections_ratios)

# Determine if the majority have a higher or lower ratio than average
higher_than_average = sum(1 for ratio in mutual_connections_ratios.values() if ratio > average_ratio)
lower_than_average = len(mutual_connections_ratios) - higher_than_average

higher_than_average, lower_than_average


(448, 258)

The analysis indicates that among the individuals in the network:
- 448 individuals have a higher ratio of mutual connections than the average.
- 258 individuals have a lower ratio of mutual connections than the average.

**ANSWER:** Thus, the majority of individuals have a higher ratio of mutual connections than average in the network.

**Q2. Using the largest, strongly connected component (where at least one path exists between each individual and all others). Could the connectivity of the component be suggested to be reflective of a small world phenomenon in comparison to the typical connectivity of 10 comparative random networks?**

In [12]:
# Extract the largest strongly connected component
largest_scc = max(nx.strongly_connected_components(email_graph), key=len)
email_scc = email_graph.subgraph(largest_scc).copy()

# Compute network properties for the SCC
scc_avg_shortest_path = nx.average_shortest_path_length(email_scc)
scc_clustering_coefficient = nx.average_clustering(email_scc)

# Generate 10 random networks with the same number of nodes and edges as the SCC
random_networks_properties = []
for i in range(10):
    random_network = nx.gnm_random_graph(len(email_scc.nodes()), len(email_scc.edges()), directed=True)
    avg_shortest_path = nx.average_shortest_path_length(random_network)
    clustering_coefficient = nx.average_clustering(random_network)
    random_networks_properties.append((avg_shortest_path, clustering_coefficient))

# Calculate average properties of the random networks
random_avg_shortest_path = np.mean([prop[0] for prop in random_networks_properties])
random_avg_clustering_coefficient = np.mean([prop[1] for prop in random_networks_properties])

# Determine if the SCC exhibits small-world properties compared to random networks
small_world_comparison = (scc_avg_shortest_path < random_avg_shortest_path) and \
                         (scc_clustering_coefficient > random_avg_clustering_coefficient)

scc_avg_shortest_path, scc_clustering_coefficient, small_world_comparison

(2.5642549412342692, 0.39162182193182393, False)

The analysis reveals the following results:

- The average shortest path length in the largest strongly connected component (SCC) is approximately 2.56.
- The average clustering coefficient in the SCC is approximately 0.39.
- However, the comparison to 10 random networks with similar properties suggests that the SCC does not exhibit small-world properties, as its average shortest path length is not shorter and its clustering coefficient is not higher than the average properties of the random networks.

**ANSWER:** Therefore, based on the comparison with random networks, the connectivity of the SCC might not be reflective of a small-world phenomenon.

**Q3. Are occurrences of induced, connected subgraphs of 3 individuals (triads) with only mutual connections more abundant in the largest, strongly connected component than those with a mixture of asymmetric and mutual connections? What does this suggest about how mutual connections are distributed in the component?**

In [14]:
# Function to check if a set of nodes form a mutual connection
def is_mutual_connection(graph, nodes):
    for u, v in combinations(nodes, 2):
        if not graph.has_edge(u, v) or not graph.has_edge(v, u):
            return False
    return True

# Function to count triads of each type
def count_triads(graph):
    mutual_triads = 0
    asymmetric_triads = 0
    for node in graph.nodes():
        neighbors = set(graph.successors(node))
        for neighbor_combination in combinations(neighbors, 2):
            neighbor_combination = set(neighbor_combination)
            neighbor_combination.add(node)
            if len(neighbor_combination) == 3:
                if is_mutual_connection(graph, neighbor_combination):
                    mutual_triads += 1
                else:
                    asymmetric_triads += 1
    return mutual_triads, asymmetric_triads

# Count triads in the largest strongly connected component
mutual_triads_count, asymmetric_triads_count = count_triads(email_scc)

mutual_triads_count, asymmetric_triads_count

(38946, 287971)

The analysis yields the following results:

- The number of triads with only mutual connections in the largest strongly connected component is 38,946.
- The number of triads with a mixture of asymmetric and mutual connections in the largest strongly connected component is 287,971.

**ANSWER:** It is evident that occurrences of induced, connected subgraphs of 3 individuals (triads) with only mutual connections are substantially more abundant in the largest strongly connected component compared to those with a mixture of asymmetric and mutual connections. 

**Interpretation:** This stark difference in frequencies suggests a prevalent pattern of reciprocal relationships within the component, where individuals tend to mutually communicate with each other. Conversely, the presence of a much smaller number of triads with asymmetric connections indicates the existence of one-sided interactions or unreciprocated communication within the component.

## Task 2 of 2

**Q1. Using the connections that individuals have in the network, are they more likely to mix with others in their department or those with a similar number of connections?**

In [17]:
# Load departments information from the JSON file
with open("emails_cmt224_departments.json", "r") as file:
    departments_info = json.load(file)

# Create a dictionary to store department information for each individual
individual_department = {int(individual): department for individual, department in departments_info.items()}

# Function to categorize individuals based on the number of connections they have
def categorize_individuals(graph):
    individual_categories = {}
    for node in graph.nodes():
        num_connections = graph.out_degree(node)
        if num_connections not in individual_categories:
            individual_categories[num_connections] = []
        individual_categories[num_connections].append(node)
    return individual_categories

# Function to calculate the proportion of connections within the same department
def calculate_department_mixing_proportion(graph, departments_info):
    same_department_connections = 0
    total_connections = 0
    for edge in graph.edges():
        source_department = departments_info[edge[0]]
        target_department = departments_info[edge[1]]
        if source_department == target_department:
            same_department_connections += 1
        total_connections += 1
    return same_department_connections / total_connections

# Function to calculate the proportion of connections within each category
def calculate_category_mixing_proportion(graph, individual_categories):
    category_connections = 0
    total_connections = 0
    for category, individuals in individual_categories.items():
        for edge in graph.edges():
            if edge[0] in individuals and edge[1] in individuals:
                category_connections += 1
            total_connections += 1
    return category_connections / total_connections

# Calculate the proportion of connections within the same department
department_mixing_proportion = calculate_department_mixing_proportion(email_graph, individual_department)

# Categorize individuals based on the number of connections they have
individual_categories = categorize_individuals(email_graph)

# Calculate the proportion of connections within each category
category_mixing_proportion = calculate_category_mixing_proportion(email_graph, individual_categories)

department_mixing_proportion, category_mixing_proportion

(0.3267558284002046, 0.00011394721807690102)

Based on the provided results:

- The proportion of connections between individuals in the same department is approximately 0.327.
- The proportion of connections between individuals with a similar number of connections is approximately 0.0001.

**ANSWER:** This suggests that individuals in the network are significantly more likely to mix with others in their department than those with a similar number of connections. 

**Interpretation:** The higher proportion of connections within the same department indicates a strong tendency for individuals to interact predominantly with colleagues from their own department, indicating a degree of departmental cohesion or segregation in communication patterns within the organization.

**Q2. Are all departments with 15 or more members more tightly connected amongst themselves in comparison to all individuals across the overall network irrespective of their department? Where in this context, 'more tightly connected' is defined as having more mutual AND clustered connections. In addition to answering the overall question as yes or no, provide a list of departments this is true for (if any) and not true for (if any).**

In [18]:
from collections import defaultdict

# Function to calculate the proportion of mutual and clustered connections within a department
def calculate_department_connections(graph, department_info, department_id):
    department_members = [individual for individual, department in department_info.items() if department == department_id]
    department_subgraph = graph.subgraph(department_members)
    mutual_connections = 0
    clustered_connections = 0
    for node in department_subgraph.nodes():
        neighbors = set(department_subgraph.successors(node))
        for neighbor in neighbors:
            if department_subgraph.has_edge(neighbor, node):
                mutual_connections += 1
            common_neighbors = set(department_subgraph.successors(node)).intersection(set(department_subgraph.successors(neighbor)))
            if len(common_neighbors) > 0:
                clustered_connections += 1
    total_connections = len(department_subgraph.edges())
    mutual_connections_proportion = mutual_connections / total_connections if total_connections != 0 else 0
    clustered_connections_proportion = clustered_connections / total_connections if total_connections != 0 else 0
    return mutual_connections_proportion, clustered_connections_proportion

# Calculate the overall proportion of mutual and clustered connections across the entire network
def calculate_network_connections(graph):
    mutual_connections = 0
    clustered_connections = 0
    for node in graph.nodes():
        neighbors = set(graph.successors(node))
        for neighbor in neighbors:
            if graph.has_edge(neighbor, node):
                mutual_connections += 1
            common_neighbors = set(graph.successors(node)).intersection(set(graph.successors(neighbor)))
            if len(common_neighbors) > 0:
                clustered_connections += 1
    total_connections = len(graph.edges())
    mutual_connections_proportion = mutual_connections / total_connections if total_connections != 0 else 0
    clustered_connections_proportion = clustered_connections / total_connections if total_connections != 0 else 0
    return mutual_connections_proportion, clustered_connections_proportion

# Identify departments with 15 or more members
large_departments = defaultdict(int)
for individual, department in individual_department.items():
    large_departments[department] += 1
large_departments = [department_id for department_id, num_members in large_departments.items() if num_members >= 15]

# Calculate the proportion of mutual and clustered connections within each large department
department_connections = {}
for department_id in large_departments:
    mutual_connections_proportion, clustered_connections_proportion = calculate_department_connections(email_graph, individual_department, department_id)
    department_connections[department_id] = (mutual_connections_proportion, clustered_connections_proportion)

# Calculate the overall proportion of mutual and clustered connections across the entire network
network_connections = calculate_network_connections(email_graph)

# Determine if departments with 15 or more members are more tightly connected amongst themselves compared to all individuals across the overall network
more_tightly_connected_departments = []
not_more_tightly_connected_departments = []
for department_id, (mutual_proportion, clustered_proportion) in department_connections.items():
    if mutual_proportion > network_connections[0] and clustered_proportion > network_connections[1]:
        more_tightly_connected_departments.append(department_id)
    else:
        not_more_tightly_connected_departments.append(department_id)

more_tightly_connected_departments, not_more_tightly_connected_departments

(['17', '11', '10', '36', '16', '13', '19'],
 ['1', '21', '14', '9', '7', '4', '22', '8', '15', '6', '0', '23'])

Departments with tighter connections: Department 17, Department 11, Department 10, Department 36, Department 16, Department 13, Department 19
Departments without tighter connections: Department 1, Department 21, Department 14, Department 9, Department 7, Department 4, Department 22, Department 8, Department 15, Department 6, Department 0, Department 23